In [6]:
import os
import json
import requests
from pydantic import BaseModel
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

client = OpenAI()

class FounderResponse(BaseModel):
    name: str

class ClassificationsResponse(BaseModel):
    company_url: str
    classifications: list[FounderResponse]

In [7]:
instructions_prompt = """
You are a helpful assistant that retrieves the names of the founders of a startup. Your task is to return the information in JSON format. Please follow these steps:

	1.	Search Process: Search the web including trusted sources such as Crunchbase, LinkedIn, or other startup databases to find the founder(s) of the specified startup.
	2.	JSON Response Structure: Return the results in a JSON object with the following fields:
	•	company_name: Name of the startup (string).
	•	founders: A list of founder objects, each containing:
        •	name: The name of the founder (string).
	3.  Verify your output if possible across different data sources. 
  4.	Handle Exceptions: If no founder information is available, return a JSON object with an error message.
"""

# DONT RUN THIS PART. ASSISTANT ALREADY INITIALIZED.

assistant = client.beta.assistants.create(
  name="Founder Names Extractor V2",
  instructions=instructions_prompt,
  model="gpt-4o-mini",
  temperature=0.1,
  response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "test_schema",
            "schema": ClassificationsResponse.model_json_schema()
        }
    }
)

print(assistant.id)

# this creates an assistant with 
# assistant_id='asst_6DlrRB4peFs8MB4sdPGtq4gm'

asst_6DlrRB4peFs8MB4sdPGtq4gm


In [9]:
def extract_founders_from_messages(messages):
    for message in messages.data:
        if message.role == 'assistant':
            for content_block in message.content:
                if content_block.type == 'text':
                    json_response = content_block.text.value
                    
                    try:
                        parsed_json = json.loads(json_response)
                        return parsed_json  # Return the parsed JSON object (as a Python dictionary)
                    except json.JSONDecodeError:
                        return {"error": "Failed to decode JSON."}
    
    return {"error": "No assistant response found."}

thread = client.beta.threads.create()

def get_response(company_name, assistants_id):
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=f"Please list the founders of this company: {company_name}"
    )

    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistants_id
    )

    if run.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )
        return extract_founders_from_messages(messages)
    
    else:
        return {"status": run.status}

response = get_response(company_name="Steerlab. Location: Paris, France", assistants_id="asst_6DlrRB4peFs8MB4sdPGtq4gm")
print(response)

{'company_name': 'Steerlab', 'founders': [{'name': 'Jean-Baptiste Dusséaux'}, {'name': 'Julien Gauthier'}]}
